# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
filename = "../WeatherPy/weather_data.csv"
weather_data = pd.read_csv(filename)

weather_data.head()


,city,latitude,longitude,temp_max_F,humid,cloud,wind_mph
0,albany,42.6001,-73.9662,68.72,96,15,4.36
1,ushuaia,-54.8000,-68.3000,37.06,93,75,0.00
2,saint-philippe,-21.3585,55.7679,72.95,73,75,15.77
3,chabahar,25.2919,60.6430,87.98,80,52,6.98
4,rikitea,-23.1203,-134.9692,71.65,65,99,24.83


In [25]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        551 non-null    object 
 1   latitude    551 non-null    float64
 2   longitude   551 non-null    float64
 3   temp_max_F  551 non-null    float64
 4   humid       551 non-null    int64  
 5   cloud       551 non-null    int64  
 6   wind_mph    551 non-null    float64
dtypes: float64(4), int64(2), object(1)
memory usage: 30.3+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [96]:
# Store latitude and longitude in locations
locations = weather_data[["latitude", "longitude"]]

# use humidity as the weight
map_weight = weather_data["humid"]

# set figure layout
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px'
}

# Plot Heatmap
fig = gmaps.figure(map_type = 'HYBRID', layout=figure_layout)



# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                weights=map_weight,
                                dissipating=False, 
                                max_intensity=200,
                                point_radius=8, 
                                opacity = 0.4,
                                gradient = [
                                            'white',
                                            'aqua',
                                            'purple'
                                            ])

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', padding='1px', width='400px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [102]:
ideal_weather_places = weather_data.loc[
    (weather_data['temp_max_F']> 81) &
    (weather_data['temp_max_F']< 85) &
    (weather_data['cloud']<40) & 
    (weather_data['wind_mph']<20)]


ideal_weather_places.reset_index(inplace = True, drop = True)
ideal_weather_places

,city,latitude,longitude,temp_max_F,humid,cloud,wind_mph
0,erzin,36.9559,36.2000,83.12,59,0,3.11
1,kadungora,-7.0899,107.8864,81.93,57,25,4.47
2,hohhot,40.8106,111.6522,84.49,42,20,6.71
3,abu samrah,35.3029,37.1841,82.92,34,0,11.99
4,durusu,41.3053,28.6767,81.54,89,0,4.00
5,yatou,37.1500,122.3833,84.11,68,29,12.12
6,shahreza,32.0339,51.8794,81.90,23,32,3.02
7,puerto colombia,10.9878,-74.9547,82.49,84,5,15.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [145]:
hotel_df = ideal_weather_places[['city', 'latitude', 'longitude']]
hotel_df = hotel_df.rename(columns = {"city":"City", "latitude":"Lat", "longitude":"Lng"})

# set up additional columns to hold information
hotel_df['Hotel Name'] = ""
hotel_df['Country'] = ""

In [150]:
# find the closest hotel to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "lodging",
    "keyword" : "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # add location to params dict, using latitude and longitude columns
    params['location'] = (f'{str(row["Lat"])},{str(row["Lng"])}')

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']

        
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
            
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")



Retrieving Results for Index 0: erzin.
Closest hotel is Artemis Otel.
------------
Retrieving Results for Index 1: kadungora.
Closest hotel is Hotel Santika Garut.
------------
Retrieving Results for Index 2: hohhot.
Closest hotel is Jinjiang Inn Hohhot South Tongdao Road.
------------
Retrieving Results for Index 3: abu samrah.
Closest hotel is Resort Safari Souria منتجع سفاري سورية.
------------
Retrieving Results for Index 4: durusu.
Closest hotel is Karam Hotel.
------------
Retrieving Results for Index 5: yatou.
Closest hotel is Founder Hotel.
------------
Retrieving Results for Index 6: shahreza.
Closest hotel is خانه مسافر اسپاد.
------------
Retrieving Results for Index 7: puerto colombia.
Closest hotel is Kasamar Hostal. Puerto Colombia. Barranquilla.
------------


In [163]:
# find the Country that the city is in
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
params = {"key": g_key, "result_type":"country"}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # add location to params dict, using latitude and longitude columns
    params['latlng'] = (f'{str(row["Lat"])},{str(row["Lng"])}')

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']

        
    try:
               
        hotel_df.loc[index, 'Country'] = results[0]['address_components'][0]['long_name']
            
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: erzin.
------------
Retrieving Results for Index 1: kadungora.
------------
Retrieving Results for Index 2: hohhot.
------------
Retrieving Results for Index 3: abu samrah.
------------
Retrieving Results for Index 4: durusu.
------------
Retrieving Results for Index 5: yatou.
------------
Retrieving Results for Index 6: shahreza.
------------
Retrieving Results for Index 7: puerto colombia.
------------


In [162]:
results[0]['address_components'][0]['long_name']

'Colombia'

In [159]:
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
params = {"key": g_key, "result_type":"country"}

params['latlng'] = '36.9559,36.2000'
response = requests.get(base_url, params=params).json()
results = response['results']
results[0]['address_components'][0]['long_name']

'Turkey'

In [164]:
hotel_df

,City,Lat,Lng,Hotel Name,Country
0,erzin,36.9559,36.2000,Artemis Otel,Turkey
1,kadungora,-7.0899,107.8864,Hotel Santika Garut,Indonesia
2,hohhot,40.8106,111.6522,Jinjiang Inn Hohhot South Tongdao Road,China
3,abu samrah,35.3029,37.1841,Resort Safari Souria منتجع سفاري سورية,Syria
4,durusu,41.3053,28.6767,Karam Hotel,Turkey
5,yatou,37.1500,122.3833,Founder Hotel,China
6,shahreza,32.0339,51.8794,خانه مسافر اسپاد,Iran
7,puerto colombia,10.9878,-74.9547,Kasamar Hostal. Puerto Colombia. Barranquilla,Colombia


In [165]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [125]:
info_box_template

'\n<dl>\n<dt>Name</dt><dd>{Hotel Name}</dd>\n<dt>City</dt><dd>{City}</dd>\n<dt>Country</dt><dd>{Country}</dd>\n</dl>\n'

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [142]:
results[0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 36.9526703, 'lng': 36.2378766},
  'viewport': {'northeast': {'lat': 36.95406332989272,
    'lng': 36.23921452989272},
   'southwest': {'lat': 36.95136367010728, 'lng': 36.23651487010728}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'name': 'Artemis Otel',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 1080,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117865296681741467490">Serhat Büyükurvaylı</a>'],
   'photo_reference': 'Aap_uEASDsiXT61n-GNPmXilVvqhHk_pnwSjrxVUrBq8sWLAA2xMKYTvN5l2Z2ZRdpYbGRCVo8-jMVWzun-qTfo-L9PwrOBoxLn467MNPqG6jIC-dfwpR7X0uqbMZ4L-d20R1C96D6yK1DnrjhcvuP7GURE6M8J9V2Tceyz3RKZa2-_IOXlA',
   'width': 1920}],
 'place_id': 'ChIJQd93zj49LxURardYAwgyzQM',
 'plus_code': {'compound_code': 'X63Q+35 Erzin, Hatay, Turkey',
  'global_code': '8G8RX63Q+35'},
 'rating': 4,
 'reference': 'ChIJQd93zj49LxURardYAwgyzQM',
 'scope': 'GOO